In [1]:
import requests
from requests.exceptions import RequestException
from pyquery import PyQuery as pq
import pandas as pd
import numpy as np
from xpinyin import Pinyin
p = Pinyin()

In [2]:
def get_one_page(url):
    '''
    获取单个页面的html
    :param url: string类型，网址
    :return: string类型，单页html文本
    '''
    try:
        headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.116 Safari/537.36'
}
        response = requests.get(url,headers=headers)
        if(response.status_code == 200):
            return response.text
        return None
    except RequestException:
        return None

In [3]:
def get_info_block(pq_doc):
    '''
    获取一个房源的未处理数据块
    :param pq_doc: pyquery对象，单页html内容
    :return: pyquery对象列表，（30个）数据块列表
    '''
    infos = pq_doc('.info.clear').items() # 得到数据块生成器
    info_block_list = []
    for info in infos:
        info_block_list.append(info)
    return info_block_list

In [4]:
def anal_house_info(info_block_list):
    '''
    将数据块列表处理成特征集
    :param info_block_list: pyquery对象，数据块列表
    :return: 房源特征集列表
    '''
    final_info_list = []
    for info in info_block_list:
        one_piece = []
        title = info('.title a').text() # 标题
        community = info('.flood .positionInfo [data-el="region"]').text() # 小区
        price = float(info('.priceInfo .totalPrice').text()[:-1]) # 房价
        average_price = float(info('.priceInfo .unitPrice span').text()[2:-4]) # 每平米均价
        ori_info = info('.address .houseInfo').text()
        more_info = ori_info.split(' | ')
        
        one_piece.append(title)
        one_piece.append(community)
        one_piece.append(price)
        one_piece.append(average_price)
        one_piece.append(more_info[0]) # 房型
        one_piece.append(float(more_info[1][:-2])) # 面积
        one_piece.append(more_info[2]) # 朝向
        one_piece.append(more_info[3]) # 装修
        one_piece.append(more_info[4]) # 楼层
        one_piece.append(more_info[-1]) # 建筑类型
        # 建造时间特征可能会缺失，需要做个判断
        if len(more_info)==6:
            # 若缺失 
            one_piece.append(np.nan)
        elif more_info[-2][0:4].isdigit() == False:
            one_piece.append(np.nan)
        else:
            one_piece.append(int(more_info[-2][0:4])) # 建楼时间
        
        final_info_list.append(one_piece)
    return final_info_list

In [5]:
def main(city,location, page):
    '''
    :param city: string类型，城市名
    :param location: string类型，区域名
    :param page: string类型，爬取页数
    '''
    final = []
    print('开始爬取！')
    pinyin_city = p.get_initials(city,'').lower() # 将城市名由中文改为首字母小写
    pinyin_location = p.get_pinyin(location,'') # 将区域由中文改为拼音
    for i in range(1,int(page)+1):
        url = 'https://{}.lianjia.com/ershoufang/{}/pg{}/'.format(pinyin_city,pinyin_location,str(i))
        pq_doc = pq(get_one_page(url))
        info_block_list = get_info_block(pq_doc)
        info_list = anal_house_info(info_block_list)
        final.extend(info_list) # 数据集列表合并
        print('已爬取第{}页'.format(i))
    print('爬取完成！')
    return final

In [7]:
if __name__ == '__main__':

    city = input('请输入城市：')
    location = input('请输入区域：')
    page = input('请输入页数：')
    
    info_list = main(city,location,page)
    
    column = ['标题','小区','价格（万元）','平米单价（元）','户型','面积（平米）','朝向','装修','楼层','建筑类型','建成时间（年）']
    df = pd.DataFrame(info_list,columns=column)
    df.to_csv('../Data/{}房源.csv'.format(location),index=False,encoding = "utf-8-sig")

请输入城市： 上海
请输入区域： 黄埔
请输入页数： 100


开始爬取！
已爬取第1页
已爬取第2页
已爬取第3页
已爬取第4页
已爬取第5页
已爬取第6页
已爬取第7页
已爬取第8页
已爬取第9页
已爬取第10页
已爬取第11页
已爬取第12页
已爬取第13页
已爬取第14页
已爬取第15页
已爬取第16页
已爬取第17页
已爬取第18页
已爬取第19页
已爬取第20页
已爬取第21页
已爬取第22页
已爬取第23页
已爬取第24页
已爬取第25页
已爬取第26页
已爬取第27页
已爬取第28页
已爬取第29页
已爬取第30页
已爬取第31页
已爬取第32页
已爬取第33页
已爬取第34页
已爬取第35页
已爬取第36页
已爬取第37页
已爬取第38页
已爬取第39页
已爬取第40页
已爬取第41页
已爬取第42页
已爬取第43页
已爬取第44页
已爬取第45页
已爬取第46页
已爬取第47页
已爬取第48页
已爬取第49页
已爬取第50页
已爬取第51页
已爬取第52页
已爬取第53页
已爬取第54页
已爬取第55页
已爬取第56页
已爬取第57页
已爬取第58页
已爬取第59页
已爬取第60页
已爬取第61页
已爬取第62页
已爬取第63页
已爬取第64页
已爬取第65页
已爬取第66页
已爬取第67页
已爬取第68页
已爬取第69页
已爬取第70页
已爬取第71页
已爬取第72页
已爬取第73页
已爬取第74页
已爬取第75页
已爬取第76页
已爬取第77页
已爬取第78页
已爬取第79页
已爬取第80页
已爬取第81页
已爬取第82页
已爬取第83页
已爬取第84页
已爬取第85页
已爬取第86页
已爬取第87页
已爬取第88页
已爬取第89页
已爬取第90页
已爬取第91页
已爬取第92页
已爬取第93页
已爬取第94页
已爬取第95页
已爬取第96页
已爬取第97页
已爬取第98页
已爬取第99页
已爬取第100页
爬取完成！
